In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from quantum_classes import *
from pauli_matrices import *

I want to:
- find combined hamiltonian
- then use calculate fidelity


I need to seperate the fidelities

## Do it for the individual qubits

In [2]:
# Create a quantum system with a single qubit
natural_freq = 10 * 10**9
driving_freq = 10 * 10**9
rabi_freq = 5 * 10**6



# CCD parameters
phi_0, epsilon_m, phase_freq, theta_m = 0, rabi_freq/4, rabi_freq, 0



# Integration parameters
tol = 10**-6
evaluation_points = 1000
evaluation_time = 10*10**-6




hamiltonian = Quantum_Hamiltonian(rabi_freq, 1, np.array([natural_freq]))
system = Quantum_System(hamiltonian, 1)


# Create a quantum system with two qubits
hamiltonian_2q = Quantum_Hamiltonian(rabi_freq, 2, np.array([natural_freq, natural_freq+10**6]))
system_2q = Quantum_System(hamiltonian_2q, 2)

Notably I think I should ditch the class structure and just have functions and arguments - im not rly using in class variables enough and its just making it harder.

In [9]:



detunings = np.linspace(0, 2*10e6, 1000)

fids_1 = []
fids_2 = []
fids_seperate = []
fids_together = []

fid_combined_Us = []

t, U1s = system.calculate_unitaries(evaluation_time, evaluation_points, ham_type='ccd_rwa',
                        driving_freq=driving_freq,
                        natural_freq =  natural_freq, phi_0=phi_0,
                        epsilon_m=epsilon_m,
                        phase_freq=phase_freq,
                        theta_m=theta_m)
fid_1 = calculate_fidelitys(np.eye(2), U1s, 2)

# H1 = hamiltonian.ccd_rwa(t = 1, driving_freq=driving_freq,natural_freq =  natural_freq, phi_0=phi_0,
#                         epsilon_m=epsilon_m,
#                         phase_freq=phase_freq,
#                         theta_m=theta_m)
for detuning in detunings:
    # H2 = hamiltonian.ccd_rwa(t = 1, driving_freq=driving_freq,natural_freq =  natural_freq+detuning, phi_0=phi_0,
    #                     epsilon_m=epsilon_m,
    #                     phase_freq=phase_freq,
    #                     theta_m=theta_m)

    # H_mult = hamiltonian_2q.ccd_rwa_multiple(t = 1, driving_freq=driving_freq,
    #                     natural_freq =  natural_freq+detuning, phi_0=phi_0,
    #                     epsilon_m=epsilon_m,
    #                     phase_freq=phase_freq,
    #                     theta_m=theta_m)


    t, U2s = system.calculate_unitaries(evaluation_time, evaluation_points, ham_type='ccd_rwa',
                        driving_freq=driving_freq,
                        natural_freq =  natural_freq+detuning, phi_0=phi_0,
                        epsilon_m=epsilon_m,
                        phase_freq=phase_freq,
                        theta_m=theta_m)
    

    
    
    fid_2 = calculate_fidelitys(np.eye(2), U2s, 2)
    # fid_seperate = fid_1*fid_2
    fid_seperate = (np.abs((6*fid_1-2)*(6*fid_2-2))+4)/20

    fids_1.append(fid_1)
    fids_2.append(fid_2)
    fids_seperate.append(fid_seperate)

    fid_combined_U_det = []
    for U1, U2 in zip(U1s, U2s):
        U = np.kron(U1, U2)
        fid_combined = calculate_fidelitys(np.eye(4), [U], 2**2)
        fid_combined_U_det.append(fid_combined[0])
    fid_combined_Us.append(fid_combined_U_det)

    natural_freqs = [natural_freq, natural_freq+detuning]
    # Recalculate the unitaries with the updated Hamiltonian

    times, Us = system_2q.calculate_unitaries(evaluation_time, evaluation_points, 
                                                ham_type='ccd_rwa_multiple', 
                                                driving_freq=driving_freq, natural_freqs =  natural_freqs,
                                                phi_0=phi_0, epsilon_m=epsilon_m, 
                                                phase_freq=phase_freq, theta_m=theta_m, coupling=0)
    
    # Calculate the fidelities for the unitaries at all times
    fid_together = calculate_fidelitys(np.eye(4), Us, 2**2)
    fids_together.append(fid_together)


In [ ]:
# Convert detunings to MHz and times to microseconds
detunings_mhz = detunings / 1e6
times_us = times * 1e6

# Create a figure with 5 subplots in a 3x2 grid
fig, ((ax1, ax2), (ax3, ax4), (ax5, _)) = plt.subplots(3, 2, figsize=(12, 15))

print(fid_combined_Us)

# Create color plots
im1 = ax1.pcolormesh(detunings_mhz, times_us, np.array(fids_1).T, shading='auto', cmap='viridis')
im2 = ax2.pcolormesh(detunings_mhz, times_us, np.array(fids_2).T, shading='auto', cmap='viridis')
im3 = ax3.pcolormesh(detunings_mhz, times_us, np.array(fids_seperate).T, shading='auto', cmap='viridis')
im4 = ax4.pcolormesh(detunings_mhz, times_us, np.array(fids_together).T, shading='auto', cmap='viridis')
im5 = ax5.pcolormesh(detunings_mhz, times_us, np.array(fid_combined_Us).T, shading='auto', cmap='viridis')


# Add colorbars
fig.colorbar(im1, ax=ax1, label='Fidelity')
fig.colorbar(im2, ax=ax2, label='Fidelity')
fig.colorbar(im3, ax=ax3, label='Fidelity')
fig.colorbar(im4, ax=ax4, label='Fidelity')
fig.colorbar(im5, ax=ax5, label='Fidelity')

# Set labels and titles
ax1.set_xlabel('Detunings (MHz)')
ax1.set_ylabel('Time (µs)')
ax1.set_title('Qubit 1 Fidelity')

ax2.set_xlabel('Detunings (MHz)')
ax2.set_ylabel('Time (µs)')
ax2.set_title('Qubit 2 Fidelity')

ax3.set_xlabel('Detunings (MHz)')
ax3.set_ylabel('Time (µs)')
ax3.set_title('Separate Fidelities (reverse engineered)')

ax4.set_xlabel('Detunings (MHz)')
ax4.set_ylabel('Time (µs)')
ax4.set_title('Multiqubit hamiltonian')

ax5.set_xlabel('Detunings (MHz)')
ax5.set_ylabel('Time (µs)')
ax5.set_title('Combined Fidelities (Tensor Product)')

# Remove the empty subplot
_.remove()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()